In [65]:
import pandas as pd
from datetime import date
from plotly import express as px, graph_objects as go
from plotly.subplots import make_subplots
import random

In [66]:

# df_superstore = pd.read_excel('../sources/Sample - Superstore.xls')
df_stateabbrevs = pd.read_csv('../sources/State abbreviations.csv')
df_superstore_geos = pd.read_csv('../sources/Superstore_with_LAT_LNG.csv')
df_combined = df_superstore_geos.merge(df_stateabbrevs, how='left', left_on='State/Province', right_on='Full Name')
df_combined['Order Date'] = pd.to_datetime(df_combined['Order Date'], origin='1899-12-30', unit='D')
df_combined['Order Year'] = df_combined['Order Date'].dt.year
df_combined['Order Month'] = df_combined['Order Date'].dt.strftime("%b")
df_combined['Order Month Year'] = df_combined['Order Month'] + ' ' + df_combined['Order Year'].astype(str)
df_combined['Ship Date'] = pd.to_datetime(df_combined['Ship Date'], origin='1899-12-30', unit='D')
df_combined['Ship Year'] = df_combined['Ship Date'].dt.year
df_combined['Ship Month'] = df_combined['Ship Date'].dt.strftime("%b")
df_combined['Ship Month Year'] = df_combined['Ship Month'] + ' ' + df_combined['Ship Year'].astype(str)
df_combined['Days to Fulfill'] = (df_combined['Ship Date'] - df_combined['Order Date']).dt.days
df_combined['Profit Per Unit'] = df_combined['Profit'] / df_combined['Quantity']

df_combined = df_combined.sort_values(by='Order Date', ascending=True)


In [67]:
def filter_df_by_datetimes(df, date_column, start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    filtered_df = df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]
    return filtered_df

In [68]:
dt_start = date(2023, 1, 1)
dt_end = date(2024, 12, 31)

df_to_chart = filter_df_by_datetimes(df_combined, 'Order Date', dt_start, dt_end)

In [83]:

fulfillment_over_time_bycategory = df_to_chart.groupby(['Order Month Year', 'Order Month', 'Order Year', 'Category'])['Sales'].sum().reset_index()
cats = ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
fulfillment_over_time_bycategory['Order Month'] = pd.Categorical(fulfillment_over_time_bycategory['Order Month'], categories=cats, ordered=True)
# fig = px.bar(sales_over_time, x='Order Month Year', y='Sales', title="Sales Over Time")
fulfillment_over_time_bycategory = fulfillment_over_time_bycategory.sort_values(by=['Order Year', 'Order Month'])
sales_over_time = df_to_chart.groupby(['Order Month Year', 'Order Month', 'Order Year'])['Sales'].sum().reset_index()
sales_over_time['Order Month'] = pd.Categorical(sales_over_time['Order Month'], categories=cats, ordered=True)
sales_over_time = sales_over_time.sort_values(by=['Order Year', 'Order Month'])

fig = px.bar(fulfillment_over_time_bycategory, x='Order Month Year', y='Sales', color='Category', barmode='group')
fig.add_scatter(
    x=sales_over_time['Order Month Year'],
    y=sales_over_time['Sales'],
    mode='lines+markers',
    name='Total Sales',
    line=dict(color='green', width=2)
)
fig.update_layout(
    title='Sales Over Time',
    xaxis_title=None,
    yaxis_title=None
)

fig.show()

In [70]:
# Group by 'Order Month Year', 'Order Month', 'Order Year', 'Segment' and sum sales
sales_over_time_bysegment = df_to_chart.groupby(
    ['Order Month Year', 'Order Month', 'Order Year', 'Segment']
)['Sales'].sum().reset_index()

# Define the order of months
cats = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
sales_over_time_bysegment['Order Month'] = pd.Categorical(
    sales_over_time_bysegment['Order Month'],
    categories=cats,
    ordered=True
)

# Sort the dataframe by 'Order Year' and 'Order Month'
sales_over_time_bysegment = sales_over_time_bysegment.sort_values(by=['Order Year', 'Order Month'])

# Calculate total sales by 'Order Month Year'
total_sales_by_month = sales_over_time_bysegment.groupby('Order Month Year')['Sales'].sum().reset_index()
total_sales_by_month.rename(columns={'Sales': 'Total Sales'}, inplace=True)

# Merge total sales back into the main dataframe
sales_over_time_bysegment = sales_over_time_bysegment.merge(total_sales_by_month, on='Order Month Year')
sales_over_time_bysegment['Percentage'] = (sales_over_time_bysegment['Sales'] / sales_over_time_bysegment['Total Sales']) * 100
# Create the subplot with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Create separate bar traces for each segment
segments = sales_over_time_bysegment['Segment'].unique()
colors = ['blue', 'orange', 'green', 'red']  # Define colors for each segment

for i, segment in enumerate(segments):
    segment_data = sales_over_time_bysegment[sales_over_time_bysegment['Segment'] == segment]
    fig.add_trace(
        go.Bar(
            x=segment_data['Order Month Year'],
            y=segment_data['Percentage'],
            name=f'{segment}',
            marker=dict(color=colors[i % len(colors)])
        ),
        secondary_y=False  # Assign to primary y-axis
    )

# Add a line for total sales (for reference)
fig.add_trace(
    go.Scatter(
        x=sales_over_time_bysegment['Order Month Year'].unique(),
        y=total_sales_by_month['Total Sales'],
        mode='lines+markers',
        name='Total Sales',
        line=dict(color='black', width=2)
    ),
    secondary_y=True  # Assign to secondary y-axis
)

# Update layout
fig.update_layout(
    title='Sales Over Time by Segment (%)',
    yaxis_title='Percentage (%)',
    yaxis2_title='Total Sales',
    barmode='relative'
)

fig.show()

In [71]:
# Create the scatter_mapbox plot
fig = px.scatter_mapbox(df_to_chart,
                        lat='LAT',
                        lon='LNG',
                        size='Sales',
                        color='Region',
                        hover_data=['City', 'State/Province'],
                        title='Sales by Region',
                        mapbox_style='open-street-map',  # Free and does not require a token
                        zoom=3,
                        center={'lat': 45, 'lon': -95}  # Center the map roughly between US and Canada
                        )
fig.show()

In [72]:
# Aggregate sales by Region, Segment, Category, and Sub-Category
grouped_df = df_to_chart.groupby(['Region', 'Segment', 'Category', 'Sub-Category'], as_index=False)['Sales'].sum()

# Create lists of unique nodes for each level
regions = grouped_df['Region'].unique().tolist()
segments = grouped_df['Segment'].unique().tolist()
categories = grouped_df['Category'].unique().tolist()
sub_categories = grouped_df['Sub-Category'].unique().tolist()

# Combine all node labels
labels = regions + segments + categories + sub_categories

# Create unique source-target pairs and values
links = []

# Region -> Segment
region_segment = grouped_df.groupby(['Region', 'Segment'], as_index=False)['Sales'].sum()
links.extend(
    zip(
        [regions.index(r) for r in region_segment['Region']],
        [len(regions) + segments.index(s) for s in region_segment['Segment']],
        region_segment['Sales']
    )
)

# Segment -> Category
segment_category = grouped_df.groupby(['Segment', 'Category'], as_index=False)['Sales'].sum()
links.extend(
    zip(
        [len(regions) + segments.index(s) for s in segment_category['Segment']],
        [len(regions) + len(segments) + categories.index(c) for c in segment_category['Category']],
        segment_category['Sales']
    )
)

# Category -> Sub-Category
category_subcategory = grouped_df.groupby(['Category', 'Sub-Category'], as_index=False)['Sales'].sum()
links.extend(
    zip(
        [len(regions) + len(segments) + categories.index(c) for c in category_subcategory['Category']],
        [len(regions) + len(segments) + len(categories) + sub_categories.index(sc) for sc in category_subcategory['Sub-Category']],
        category_subcategory['Sales']
    )
)

# Separate source, target, and values
source_indices, target_indices, values = zip(*links)

# Create the Sankey diagram
import plotly.graph_objects as go

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=list(source_indices),
        target=list(target_indices),
        value=list(values)
    )
))

fig.update_layout(title_text="Sales Sankey", font_size=10)
fig.show()


In [73]:
fulfillment_trend = df_to_chart.groupby(['Order Month Year', 'Region'], as_index=False)['Days to Fulfill'].mean()
fig = px.box(
    fulfillment_trend,
    x='Region',
    y='Days to Fulfill',
    points='all',  # Show all points to highlight outliers
    title='Fulfillment Times by Region',
    hover_data=['Region', 'Order Month Year', 'Days to Fulfill'],
)

fig.show()

In [89]:
fulfillment_over_time_bycategory = df_to_chart.groupby(['Order Month Year', 'Order Month', 'Order Year', 'Category'], as_index=False)['Days to Fulfill'].mean()
# sales_over_time_bycategory = df_to_chart.groupby(['Order Month Year', 'Order Month', 'Order Year', 'Category'])['Sales'].sum().reset_index()
cats = ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
fulfillment_over_time_bycategory['Order Month'] = pd.Categorical(fulfillment_over_time_bycategory['Order Month'], categories=cats, ordered=True)
# fig = px.bar(sales_over_time, x='Order Month Year', y='Sales', title="Sales Over Time")
fulfillment_over_time_bycategory = fulfillment_over_time_bycategory.sort_values(by=['Order Year', 'Order Month'])
fulfillment_over_time = df_to_chart.groupby(['Order Month Year', 'Order Month', 'Order Year'])['Days to Fulfill'].mean().reset_index()
fulfillment_over_time['Order Month'] = pd.Categorical(fulfillment_over_time['Order Month'], categories=cats, ordered=True)
fulfillment_over_time = fulfillment_over_time.sort_values(by=['Order Year', 'Order Month'])

fig = px.bar(fulfillment_over_time_bycategory, x='Order Month Year', y='Days to Fulfill', color='Category', barmode='group')
fig.add_scatter(
    x=fulfillment_over_time['Order Month Year'],
    y=fulfillment_over_time['Days to Fulfill'],
    mode='lines+markers',
    name='Avg Days to Fulfill',
    line=dict(color='green', width=2)
)
fig.update_layout(
    title='Avg Days to Fulfill Over Time',
    xaxis_title=None,
    yaxis_title=None
)

fig.show()

In [74]:
top_sales = df_to_chart.groupby(['Product Name', 'Category', 'Sub-Category'], as_index=False)['Sales'].sum()
top_sales_sorted = top_sales.sort_values(by='Sales', ascending=False).head(15)

# Create the bar chart
fig = px.bar(
    top_sales_sorted,
    x='Sales',
    y='Product Name',
    orientation='h',  # Horizontal bar chart
    color='Sub-Category',
    title='Top Products by Sales',
    labels={'Sales': 'Total Sales', 'Product Name': 'Product Name'},
    text='Sales'
)

# Adjust layout for better readability
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(yaxis=dict(categoryorder='total ascending'))  # Ensure sorting by total sales


fig.show()

In [75]:
top_profit_per_unit = df_to_chart.groupby(['Product Name', 'Category', 'Sub-Category'], as_index=False)['Profit Per Unit'].mean()
top_profit_per_unit_sorted = top_profit_per_unit.sort_values(by='Profit Per Unit', ascending=False).head(15)

# Create the bar chart
fig = px.bar(
    top_profit_per_unit_sorted,
    x='Profit Per Unit',
    y='Product Name',
    orientation='h',  # Horizontal bar chart
    color='Sub-Category',
    title='Top Products by Sales Per Unit',
    labels={'Profit Per Unit': 'Total Profit Per Unit', 'Product Name': 'Product Name'},
    text='Profit Per Unit'
)

# Adjust layout for better readability
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(yaxis=dict(categoryorder='total ascending'))  # Ensure sorting by total sales


fig.show()

In [76]:
def chart_triple_sankey(df_to_chart, left_col, middle_col, right_col, sum_col):
    grouped_df = df_to_chart.groupby([left_col, middle_col, right_col], as_index=False)[sum_col].sum()
    lefts = (
        grouped_df.groupby(left_col, as_index=False)[sum_col]
        .sum()
        .sort_values(by=sum_col, ascending=False)[left_col]
        .tolist()
    )
    middles = (
        grouped_df.groupby(middle_col, as_index=False)[sum_col]
        .sum()
        .sort_values(by=sum_col, ascending=False)[middle_col]
        .tolist()
    )
    rights = (
        grouped_df.groupby(right_col, as_index=False)[sum_col]
        .sum()
        .sort_values(by=sum_col, ascending=False)[right_col]
        .tolist()
    )
    labels = lefts + middles + rights

    # Generate consistent colors based on node position
    max_nodes = max(len(lefts), len(middles), len(rights))
    node_colors = ['hsl({}, 70%, 50%)'.format(int(i * 360 / max_nodes)) for i in range(max_nodes)]
    node_color_map = {label: node_colors[i % max_nodes] for i, label in enumerate(lefts)}
    node_color_map.update({label: node_colors[i % max_nodes] for i, label in enumerate(middles)})
    node_color_map.update({label: node_colors[i % max_nodes] for i, label in enumerate(rights)})
    node_colors_combined = [node_color_map[label] for label in labels]
    links = []

    # Lefts -> Middles
    left_middle_segment = grouped_df.groupby([left_col, middle_col], as_index=False)[sum_col].sum()
    links.extend(
        zip(
            [lefts.index(l) for l in left_middle_segment[left_col]],
            [len(lefts) + middles.index(m) for m in left_middle_segment[middle_col]],
            left_middle_segment[sum_col]
        )
    )
    # Middles -> Rights
    middle_right_segment = grouped_df.groupby([middle_col, right_col], as_index=False)[sum_col].sum()
    links.extend(
        zip(
            [len(lefts) + middles.index(m) for m in middle_right_segment[middle_col]],
            [len(lefts) + len(middles) + rights.index(r) for r in middle_right_segment[right_col]],
            middle_right_segment[sum_col]
        )
    )
    source_indices, target_indices, values = zip(*links)
    hover = '%{source.label} / %{target.label}<br>' + sum_col + ': %{value}<extra></extra>'

    fig = go.Figure(go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
            color=node_colors_combined
        ),
        link=dict(
            source=list(source_indices),
            target=list(target_indices),
            value=list(values),
            hovertemplate=hover
        )
    ))
    fig.update_layout(title_text=f"{sum_col} Sankey", font_size=10)

    return fig



In [77]:
fig = chart_triple_sankey(df_to_chart, 'Region', 'Category', 'Segment', 'Profit')
fig.show()

In [78]:
fig = chart_triple_sankey(df_to_chart, 'Category', 'Segment', 'Region', 'Profit')
fig.show()

In [79]:
def boxplot_per_column(df_to_chart, col_groupby, col_avgby, col_figuresby):
    figs = []
    df_grouped = df_to_chart.groupby(col_groupby, as_index=False)[col_avgby].mean()
    df_grouped = df_grouped.sort_values(by=col_groupby, ascending=True)

    for i in df_grouped[col_figuresby].unique():
        cur_figuresby = df_grouped[df_grouped[col_figuresby] == i]
        hov_dat = []
        for j in col_groupby:
            hov_dat.append(f'{j}')
        hov_dat.append(f'{col_avgby}')
        figs.append(
            px.box(
                cur_figuresby,
                x=col_figuresby,
                y=col_avgby,
                points='all',  # Show all points to highlight outliers
                title=f'{col_avgby} by {i}',
                hover_data=hov_dat,
                notched=True,
            )
        )
    return figs

In [80]:
# df_profit_by_category = df_to_chart.groupby(['Order Month Year', 'Category'], as_index=False)['Profit'].mean()

for i in boxplot_per_column(df_to_chart, ['Category', 'Sub-Category'], 'Profit', 'Category'):
    i.show()